In [7]:
# 6.2 데이터를 csv로 저장

# 우선 csv 모듈로 파일을 열고 쓰고 닫아보자. 바로 csv 파일로 만들어 줄 것이다. 

import csv 

csvFile = open('Ch6_test.csv', 'w+') # 파일이 없다 -> 새로 만든다. / 파일이 이미 있다 -> 덮어쓴다. (조심)

try:
    writer = csv.writer(csvFile)  # csv.writer() 함수로 연 파일을 writer 객체에 담는다.
    writer.writerow(('number', 'number plus 2', 'number times 2'))   # 한줄 쓸 때는 이와 같이 .writerow()
    for i in range(10):
        writer.writerow((i, i+2, i*2))

finally:
    csvFile.close()  # 오픈한 파일은 꼭 닫아준다.
    
# 당연히 workplace -> web_scraping 디렉토리에서 확인할 수 있다.

In [6]:
# HTML 테이블을 가져와서 csv 파일을 만들어보자.
# https://en.wikipedia.org/wiki/Comparison_of_text_editers 항목의 복잡한 HTML 테이블을 저장해보자.

import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://en.wikipedia.org/wiki/Comparison_of_text_editors')
bs = BeautifulSoup(html, 'html.parser')
# 현재 페이지의 테이블의 첫 번째를 객체로 만든다.
table = bs.findAll('table', {'class':'wikitable'})[0] 
# 근데 분명 html 상에는 wikitable이라는 클래스가 없는데, 어떻게 코드가 정상적으로 작동하는 지 모르겠다.
# 사실 저장된 파일도 어디에 위치한 것인지... 우선 방법만 터득하자. 
rows = table.findAll('tr')

csvFile = open('Ch6_editors.csv', 'wt+', encoding='UTF-8')
writer = csv.writer(csvFile)
try:
    for row in rows:
        csvRow = []
        for cell in row.findAll(['td','th']):
            csvRow.append(cell.get_text())
            writer.writerow(csvRow)
finally:
    csvFile.close()
    
"""
파일을 오픈할 때, UnicodeEncodeError가 나는 경우가 있는데, 
파이썬이 기본이 UTF-8이 아니라서 비-영어권 사용자에게 생기는 문제이다. 

이때에는 위 코드와 같이, endcoding= 'UTF-8' 을 덧붙여 주면 된다.
"""

# class tr td th 등 html 구조에 대해서 조금 더 공부해 보자.

"\n파일을 오픈할 때, UnicodeEncodeError가 나는 경우가 있는데, \n파이썬이 기본이 UTF-8이 아니라서 비-영어권 사용자에게 생기는 문제이다. \n\n이때에는 위 코드와 같이, endcoding= 'UTF-8' 을 덧붙여 주면 된다.\n"

In [ ]:
# 6.3 MySQL

# 마이시퀄을 사용해 데이터를 관리해보자!
"""
자. 설치를 완료하고, 간단한 명령어 사용법을 익혔으니 바로 PyMySQL 패키지를 통해 데이터베이스를 이용해보자.
"""

In [3]:
import pymysql
conn = pymysql.connect(host='127.0.0.1',user='root',passwd='3030',db='mysql')
# 모듈을 이용해 mysql에 접속한 '연결 객체'를 생성하는 것이다. 이때 당연히 아이디 비밀번호가 들어가게 된다.

cur = conn.cursor() # 연결 객체 이외에도,'커서 객체'를 만들어 준다.

"""
연결 객체는 데이터 베이스 연결에 관여하면서 그 외에도 데이터베이스에 정보를 보내고 롤백을 처리하며
새 커서 객체를 만드는 역할을 한다.

연결 객체 하나에는 커서가 여러 개가 있을 수 있는데, 
커서 객체는 어떤 데이터베이스를 사용 중인지와 같은 '상태' 정보를 추적하는 역할을 한다. 
또한 마지막에 실행한 쿼리 결과도 가지고 있는데, cur.fetchone()은 커서에서 함수를 호출해 이 정보에 접근한다.

당연히 커서와 연결 사용을 마치면 이들을 닫아야 한다. 연결 누수 현상이 발생할 수 있기 때문이다.
"""

cur.execute("USE scraping")
cur.execute("SELECT * FROM pages WHERE id = 2")
print(cur.fetchone())
cur.close()
conn.close()

"""
마치 MySQL cmd에서 명령어를 친 것과 같이, 결과가 출력된다.
cur.fetchone()이 말했듯 이 정보를 가지고 올 수 있다.

참고로 책과 비교하면서 복습 중이라면, 참고할 것은 id = 2인 이유는 한번 id = 1 을 만들었다가 삭제했기 때문이다.
삭제되었더라도 id는 밑에서부터 채워지지 않는다. (auto_increment)
"""

# 컴활딸 때 공부했던, ACCESS랑 명령어가 본질적으로 동일하다. 이런 식으로 도움이 되는구나 싶다. 

(2, 'Test page title', 'This is some test page content. It can be up to 10,000 characters long.', datetime.datetime(2022, 3, 25, 15, 15, 43))


In [ ]:
# 자. 이제 본격적으로 스크레이핑 결과를 데이터베이스에 저장하는 일을 해보도록 하자. 